In [2]:
%cd ..

c:\Users\HP\OneDrive - University of Moratuwa\Desktop\TMS-Project\TMS-Backend


In [ ]:
import pandas as pd 
from src.database.database import engine
from config.settings import Settings
from src.utils.master_utils import get_osrm_data
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

class MasterManager:
    def __init__(self, configs=None):
        self.config = configs
        if self.config is None:
            self.config = Settings()
            
        self.gps_df = self._load_gps()
        
        self.distance_matrix_path = self.config.distance_matrix_file_path
        self.duration_matrix_path = self.config.duration_matrix_file_path
        
    def _load_gps(self):
        df = pd.read_sql("SELECT * FROM master_gps", engine)
        gps_csv_path = self.config.master_folder_path / self.config.gps_file_name
        df.to_csv(gps_csv_path, index=False)
        return df
    
    # def update_matrix(self):
    #     start = time.time()
    #     if not self.distance_matrix_path.exists():
    #         # gps locations and its shop codes
    #         shop_codes = self.gps_df['shop_code'].values
    #         coords = self.gps_df[['latitude', 'longitude']].values
    #         # create  matrixes
    #         distance_matrix = np.zeros((len(shop_codes), len(shop_codes)))
    #         duration_matrix = np.zeros((len(shop_codes), len(shop_codes)))
    #         for i in range(len(shop_codes)):
    #             for j  in range(i, len(shop_codes)):
    #                 if i == j:
    #                     distance_matrix[i][j] = 0
    #                     duration_matrix[i][j] = 0
    #                     continue
    #                 origin = tuple(coords[i])
    #                 destination = tuple(coords[j])
                
    #                 _, distance, duration = get_osrm_data(origin, destination)
                    
    #                 distance_matrix[i][j] = round(distance, 3)
    #                 distance_matrix[j][i] = round(distance, 3)
                    
    #                 duration_matrix[i][j] = round(duration, 2)
    #                 duration_matrix[j][i] = round(duration, 2)
    #     print(time.time()-start)
    #     print(distance_matrix)
    #     print(duration_matrix)

    def _get_osrm_thread(self, origin, destination):
        _, distance, duration = get_osrm_data(origin, destination)
        return round(distance, 3), round(duration, 2)

    def update_matrix(self):
        start = time.time()
        if not self.distance_matrix_path.exists():
            # gps locations and its shop codes
            shop_codes = self.gps_df['shop_code'].values
            coords = self.gps_df[['latitude', 'longitude']].values
            # create matrixes
            distance_matrix = np.zeros((len(shop_codes), len(shop_codes)))
            duration_matrix = np.zeros((len(shop_codes), len(shop_codes)))
            
            # **THREAD MAGIC: REPLACE YOUR LOOP WITH THESE 6 LINES**
            with ThreadPoolExecutor(max_workers=50) as executor:
                futures = {}
                for i in range(len(shop_codes)):
                    for j in range(i+1, len(shop_codes)):  # Skip i==j & duplicates
                        origin = tuple(coords[i])
                        destination = tuple(coords[j])
                        future = executor.submit(self._get_osrm_thread, origin, destination)
                        futures[future] = (i, j)
                
                for future in as_completed(futures):
                    i, j = futures[future]
                    distance, duration = future.result()
                    distance_matrix[i][j] = distance
                    distance_matrix[j][i] = distance
                    duration_matrix[i][j] = duration
                    duration_matrix[j][i] = duration
                    
            distance_df = pd.DataFrame(distance_matrix, columns=shop_codes, index=list(shop_codes))
            duration_df = pd.DataFrame(duration_matrix, columns=shop_codes, index=list(shop_codes))
            
            distance_df.to_csv(self.distance_matrix_path)
            duration_df.to_csv(self.duration_matrix_path)
            self.gps_df.to_csv(self.config.master_folder_path/'previous_gps.csv', index=False)
        print(time.time() - start)     
        print(distance_matrix)
        print(duration_matrix)

In [16]:
manager = MasterManager()

In [17]:
list(manager.gps_df['shop_code'].values)

['0', '3', '4', '5']

In [18]:
manager.update_matrix()

20.282389640808105
[[ 0.     7.41  24.93  16.722]
 [ 7.41   0.    18.816 10.303]
 [24.93  18.816  0.    11.478]
 [16.722 10.303 11.478  0.   ]]
[[ 0.    9.13 32.39 21.23]
 [ 9.13  0.   25.71 14.04]
 [32.39 25.71  0.   15.92]
 [21.23 14.04 15.92  0.  ]]


In [119]:
manager.update_matrix()

6.137975692749023
[[ 0.     7.41  24.93  16.722]
 [ 7.41   0.    18.816 10.303]
 [24.93  18.816  0.    11.478]
 [16.722 10.303 11.478  0.   ]]
[[ 0.    9.13 32.39 21.23]
 [ 9.13  0.   25.71 14.04]
 [32.39 25.71  0.   15.92]
 [21.23 14.04 15.92  0.  ]]


In [106]:
df = manager.gps_df

In [109]:
from itertools import combinations

shop_codes = df['shop_code'].values
coords = df[['latitude', 'longitude']].values
n = len(shop_codes)

pairs = [(coords[i], coords[j]) for i, j in combinations(range(n), 2)]

In [110]:
pairs

[(array([ 7.0038321, 79.9394804]), array([ 6.9906677, 79.8931709])),
 (array([ 7.0038321, 79.9394804]), array([ 6.8409891, 79.9017187])),
 (array([ 7.0038321, 79.9394804]), array([ 6.9175873, 79.8585189])),
 (array([ 6.9906677, 79.8931709]), array([ 6.8409891, 79.9017187])),
 (array([ 6.9906677, 79.8931709]), array([ 6.9175873, 79.8585189])),
 (array([ 6.8409891, 79.9017187]), array([ 6.9175873, 79.8585189]))]

In [ ]:
import pandas as pd 
from src.database.database import engine
from config.settings import Settings
import numpy as np
import asyncio
import requests  # For async session
from itertools import combinations  # For pairs if needed

# **UPDATE get_osrm_data TO ASYNC IN utils/master_utils.py**
# Replace your sync get_osrm_data with this async version:
async def get_osrm_data_async(session, origin, destination):
    """
    ASYNC version - Get distance and duration
    """
    osrm_base_url = "http://router.project-osrm.org/route/v1/car"
    url = f"{osrm_base_url}/{origin[1]},{origin[0]};{destination[1]},{destination[0]}?overview=false"
    
    try:
        response = await session.get(url, timeout=10)
        if response.status_code == 200:
            data = response.json()
            if "routes" in data and len(data["routes"]) > 0:
                distance = data["routes"][0]["distance"] / 1000  # km
                duration = data["routes"][0]["duration"] / 60  # min
                return round(distance, 3), round(duration, 2)
        return 0.0, 0.0
    except Exception as e:
        print(f"Error: {e}")
        return 0.0, 0.0

class MasterManager:
    def __init__(self, configs=None):
        self.config = configs
        if self.config is None:
            self.config = Settings()
            
        self.gps_df = self._load_gps()
        
        self.distance_matrix_path = self.config.distance_matrix_file_path
        self.duration_matrix_path = self.config.duration_matrix_file_path
        
    def _load_gps(self):
        df = pd.read_sql("SELECT * FROM master_gps", engine)
        gps_csv_path = self.config.master_folder_path / self.config.gps_file_name
        df.to_csv(gps_csv_path, index=False)
        return df
    
    # **ASYNC HELPER METHOD (ADD THIS)**
    async def _fetch_osrm(self, session, origin, destination):
        # from src.utils.master_utils import get_osrm_data_async
        return await get_osrm_data_async(session, origin, destination)
    
    def update_matrix(self):
        if not self.distance_matrix_path.exists():
            # gps locations and its shop codes
            shop_codes = self.gps_df['shop_code'].values
            coords = self.gps_df[['latitude', 'longitude']].values
            # create matrixes
            distance_matrix = np.zeros((len(shop_codes), len(shop_codes)))
            duration_matrix = np.zeros((len(shop_codes), len(shop_codes)))
            
            # **ASYNC MAGIC: REPLACE YOUR LOOP**
            print(f"🚀 ASYNC Launching {len(shop_codes)*(len(shop_codes)-1)//2} parallel calls...")
            asyncio.run(self._async_update(coords, distance_matrix, duration_matrix))
            
            print(distance_matrix)
            print(duration_matrix)
            
    
    # **ADD THIS ASYNC METHOD**
    async def _async_update(self, coords, distance_matrix, duration_matrix):
        async with requests.Session() as session:
            tasks = []
            n = len(coords)
            for i in range(n):
                for j in range(i + 1, n):  # Upper triangle only
                    origin = tuple(coords[i])
                    destination = tuple(coords[j])
                    tasks.append(self._fetch_osrm(session, origin, destination))
            
            results = await asyncio.gather(*tasks, return_exceptions=True)
            
            # Fill results back
            idx = 0
            for i in range(n):
                for j in range(i + 1, n):
                    if isinstance(results[idx], Exception):
                        dist, dur = 0.0, 0.0
                    else:
                        dist, dur = results[idx]
                    distance_matrix[i][j] = dist
                    distance_matrix[j][i] = dist
                    duration_matrix[i][j] = dur
                    duration_matrix[j][i] = dur
                    idx += 1
            
            # Diagonals
            np.fill_diagonal(distance_matrix, 0)
            np.fill_diagonal(duration_matrix, 0)

In [22]:
manager = MasterManager()
manager.update_matrix()

🚀 ASYNC Launching 6 parallel calls...


RuntimeError: asyncio.run() cannot be called from a running event loop